In [ ]:
import pandas as pd
import numpy as np
import mne
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import re

In [ ]:
labels_df = pd.read_csv("C:/Users/annbu/affect/labels.csv")

In [ ]:
def extract_features(fif_file):
    raw = mne.io.read_raw_fif(fif_file, preload=True)
    data, _ = raw.get_data(return_times=True)

    features = []
    for channel in data:
        # Статистичні характеристики
        mean_val = np.mean(channel)
        std_val = np.std(channel)

        # Потужності в частотних діапазонах
        psd, freqs = mne.time_frequency.psd_array_welch(channel[None, :], sfreq=raw.info['sfreq'], fmin=1, fmax=40)
        #delta_power = psd[:, (freqs >= 1) & (freqs < 4)].mean() # немає відношення так написано в статті
        theta_power = psd[:, (freqs >= 4) & (freqs < 8)].mean()
        alpha_power = psd[:, (freqs >= 8) & (freqs < 13)].mean()
        beta_power = psd[:, (freqs >= 13) & (freqs < 30)].mean()
        gamma_power = psd[:, (freqs >= 30) & (freqs < 40)].mean()

        # Додати всі характеристики
        features.extend([mean_val, std_val, theta_power, alpha_power, beta_power, gamma_power])

    return features

In [ ]:
X = []  # всі feature
y_valence = []  # валентність
y_arousal = []  # збудженість
y_emotion = []  # емоція

for index, row in labels_df.iterrows():
    try:
        # Витягуємо частину назви файлу (SUBXXVIDXX)
        match_filename_part = re.search(r"(SUB\d+VID\d+)", row['experiment_code'], re.IGNORECASE)
        filename_part = match_filename_part.group(1) if match_filename_part else "UNKNOWN"

        print(filename_part)

        print(row['experiment_code'])

        file_path = f"processed/{row['experiment_code']}_eeg.fif"
        features = extract_features(file_path)

        # витягуємо номер суб'єкта і переводимо в інт (наприклад, "SUB01" -> 1)
        match = re.search(r"sub(\d+)", row['experiment_code'], re.IGNORECASE)
        subject_number = int(match.group(1)) if match else -1  # якщо не знайшли, ставимо -1

        features.append(subject_number)

        X.append(row['experiment_code'])
        X.append(features)

        y_valence.append(row['valence'])
        y_arousal.append(row['arousal'])
        y_emotion.append(row['emotion'])
    except FileNotFoundError:
        print(f"Файл не знайдено: {file_path}. Пропускаю.")
        continue
    except Exception as e:
        print(f"Помилка з файлом {row['experiment_code']}: {e}")
        continue

UNKNOWN
SUB01TST04
Opening raw data file processed/SUB01TST04_eeg.fif...
Файл не знайдено: processed/SUB01TST04_eeg.fif. Пропускаю.
UNKNOWN
SUB01TST03
Opening raw data file processed/SUB01TST03_eeg.fif...
Файл не знайдено: processed/SUB01TST03_eeg.fif. Пропускаю.
UNKNOWN
SUB01TST02
Opening raw data file processed/SUB01TST02_eeg.fif...
Файл не знайдено: processed/SUB01TST02_eeg.fif. Пропускаю.
SUB01VID25
SUB01VID25
Opening raw data file processed/SUB01VID25_eeg.fif...
    Range : 0 ... 30976 =      0.000 ...   121.000 secs
Ready.
Reading 0 ... 30976  =      0.000 ...   121.000 secs...
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
Effective window size : 1.000 (s)
E

In [ ]:
X = pd.read_csv("data.csv")  # всі feature
y_valence = pd.read_csv("valence.csv")  # валентність
y_arousal = pd.read_csv("arousal.csv")  # збудженість
y_emotion = pd.read_csv("emotion.csv")  # емоція

In [ ]:
y_valence= y_valence['valence'].tolist()
y_arousal= y_arousal['arousal'].tolist()

In [ ]:
X = X.drop('name_file', axis=1)

In [ ]:
channels = [f"Ch{i+1}" for i in range(14)]
feature_names = ["Mean", "Std Dev", "Theta Power", "Alpha Power", "Beta Power", "Gamma Power"]

column_names = [f"{ch}_{feat}" for ch in channels for feat in feature_names] + ['experiment_code'] + ['name_file']

df = pd.DataFrame(X, columns=column_names)

In [ ]:
df.to_csv('data.csv', index=False)

In [ ]:
df

,Ch1_Mean,Ch1_Std Dev,Ch1_Theta Power,Ch1_Alpha Power,Ch1_Beta Power,Ch1_Gamma Power,Ch2_Mean,Ch2_Std Dev,Ch2_Theta Power,Ch2_Alpha Power,...,Ch13_Beta Power,Ch13_Gamma Power,Ch14_Mean,Ch14_Std Dev,Ch14_Theta Power,Ch14_Alpha Power,Ch14_Beta Power,Ch14_Gamma Power,experiment_code,name_file
0,-0.007856,12.848280,6.376499,2.953373,1.182572,0.902507,0.032383,13.196769,5.805588,2.605096,...,1.548722,1.127173,0.034937,15.305706,8.174091,4.119650,2.271719,1.697121,1,SUB01VID25
1,-0.029390,15.368111,10.126835,3.402572,1.055245,0.734178,-0.100979,17.402255,11.545784,3.135453,...,1.980287,1.770854,-0.005397,17.535070,11.545706,4.851728,2.409748,2.075711,1,SUB01VID24
2,0.006996,16.017913,11.412445,3.818720,1.069875,0.635234,0.101066,19.616918,12.825493,3.197154,...,1.350474,0.833465,-0.021813,17.355534,12.593529,5.084382,1.968709,1.260414,1,SUB01VID23
3,-0.027253,16.642966,9.899070,3.429128,0.973598,0.681272,-0.038493,18.239224,10.016335,2.955328,...,1.399758,0.940101,0.009967,18.450414,12.909608,4.748752,2.059875,1.424335,1,SUB01VID21
4,-0.018234,11.732200,6.696972,2.959700,0.806932,0.467701,-0.031063,14.720739,8.055340,2.666162,...,1.316976,0.632615,0.050999,14.427890,8.366914,3.664933,1.693697,1.025174,1,SUB01VID20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,0.093502,16.977695,10.024260,4.188634,0.921608,0.563430,0.124918,17.093031,7.998522,3.265881,...,1.148288,0.619085,-0.069257,19.580092,16.294567,8.159360,1.425241,0.819023,30,SUB30VID20
678,0.051983,22.143175,21.379737,10.710927,1.022601,0.392993,0.067005,19.821881,15.314154,6.700195,...,1.161460,0.527226,0.039255,26.755658,34.254481,17.412065,1.670053,0.631565,30,SUB30VID21
679,-0.026329,18.029723,15.205692,12.050139,0.950048,0.386221,-0.056839,17.260305,10.613487,6.716624,...,1.315870,0.641469,-0.007464,22.331778,25.851547,21.663111,1.568391,0.586981,30,SUB30VID23
680,-0.013981,20.094219,13.989340,8.056362,0.902629,0.408105,0.025374,20.297761,11.081365,5.807567,...,1.229555,0.779277,-0.087330,25.290151,22.638414,14.461735,1.437180,0.672421,30,SUB30VID24


In [ ]:
X = np.array(X)

In [ ]:
y_valence_df = pd.DataFrame(y_valence, columns=['valence'])
y_valence_df.to_csv('valence.csv', index=False)

y_arousal_df = pd.DataFrame(y_arousal, columns=['arousal'])
y_arousal_df.to_csv('arousal.csv', index=False)

y_emotion_df = pd.DataFrame(y_emotion, columns=['emotion'])
y_emotion_df.to_csv('emotion.csv', index=False)

In [ ]:
# для кодування назв емоцій
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
categories = y_emotion
y_emotion_encoded = le.fit_transform(categories)
y_emotion_encoded
# decoded = le.inverse_transform(encoded)  # Повертаємо до текстових міток
# print(decoded)


C:\Users\annbu\AppData\Roaming\Python\Python312\site-packages\sklearn\preprocessing\_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([6, 5, 5, 0, 1, 4, 1, 5, 2, 2, 3, 5, 5, 2, 3, 4, 2, 1, 5, 4, 0, 3,
       1, 1, 3, 0, 4, 5, 1, 2, 4, 3, 1, 3, 4, 0, 2, 5, 5, 1, 4, 1, 0, 5,
       5, 4, 4, 5, 4, 2, 2, 4, 2, 5, 2, 0, 3, 5, 4, 6, 3, 4, 6, 1, 5, 4,
       1, 3, 1, 1, 3, 1, 4, 5, 1, 6, 4, 4, 6, 3, 5, 4, 0, 6, 5, 6, 4, 1,
       0, 4, 5, 6, 3, 4, 2, 2, 4, 2, 5, 0, 2, 3, 5, 5, 6, 3, 0, 4, 3, 1,
       5, 4, 1, 3, 1, 2, 3, 1, 2, 5, 1, 6, 4, 3, 2, 5, 5, 6, 0, 2, 5, 2,
       4, 1, 0, 5, 5, 6, 4, 5, 1, 0, 6, 6, 6, 5, 2, 1, 4, 5, 3, 1, 3, 4,
       6, 1, 5, 3, 1, 3, 0, 1, 3, 1, 4, 5, 1, 6, 4, 3, 2, 5, 4, 3, 0, 2,
       0, 2, 6, 2, 1, 4, 6, 5, 6, 5, 3, 6, 2, 6, 6, 5, 2, 0, 3, 5, 5, 2,
       4, 4, 6, 1, 5, 3, 1, 3, 1, 1, 3, 4, 4, 5, 2, 6, 5, 2, 4, 4, 4, 4,
       4, 4, 6, 1, 5, 2, 1, 4, 1, 5, 2, 0, 4, 2, 3, 6, 4, 4, 6, 1, 5, 4,
       0, 3, 1, 1, 3, 0, 4, 5, 1, 6, 4, 3, 1, 3, 5, 3, 0, 2, 0, 6, 4, 1,
       6, 3, 5, 3, 6, 5, 4, 2, 1, 4, 2, 5, 2, 0, 4, 5, 3, 2, 3, 4, 2, 1,
       5, 4, 0, 3, 1, 1, 3, 2, 4, 5, 1, 2, 4, 3, 1,

In [ ]:
decoded = le.inverse_transform(y_emotion_encoded)  # Повертаємо до текстових міток
print(decoded)

['surprise' 'sad' 'sad' 'angry' 'disgust' 'neutral' 'disgust' 'sad' 'fear'
 'fear' 'happy' 'sad' 'sad' 'fear' 'happy' 'neutral' 'fear' 'disgust'
 'sad' 'neutral' 'angry' 'happy' 'disgust' 'disgust' 'happy' 'angry'
 'neutral' 'sad' 'disgust' 'fear' 'neutral' 'happy' 'disgust' 'happy'
 'neutral' 'angry' 'fear' 'sad' 'sad' 'disgust' 'neutral' 'disgust'
 'angry' 'sad' 'sad' 'neutral' 'neutral' 'sad' 'neutral' 'fear' 'fear'
 'neutral' 'fear' 'sad' 'fear' 'angry' 'happy' 'sad' 'neutral' 'surprise'
 'happy' 'neutral' 'surprise' 'disgust' 'sad' 'neutral' 'disgust' 'happy'
 'disgust' 'disgust' 'happy' 'disgust' 'neutral' 'sad' 'disgust'
 'surprise' 'neutral' 'neutral' 'surprise' 'happy' 'sad' 'neutral' 'angry'
 'surprise' 'sad' 'surprise' 'neutral' 'disgust' 'angry' 'neutral' 'sad'
 'surprise' 'happy' 'neutral' 'fear' 'fear' 'neutral' 'fear' 'sad' 'angry'
 'fear' 'happy' 'sad' 'sad' 'surprise' 'happy' 'angry' 'neutral' 'happy'
 'disgust' 'sad' 'neutral' 'disgust' 'happy' 'disgust' 'fear' 'happy

In [ ]:
# розділення на тренувальні та тестові дані
X_train_valence, X_test_valence, y_train_valence, y_test_valence = train_test_split(X, y_valence, test_size=0.2, random_state=42)
X_train_arousal, X_test_arousal, y_train_arousal, y_test_arousal = train_test_split(X, y_arousal, test_size=0.2, random_state=42)
X_train_emotion, X_test_emotion, y_train_emotion, y_test_emotion = train_test_split(X, y_emotion, test_size=0.2, random_state=42)

In [ ]:
# якщо треба, спробувати смоук
from imblearn.over_sampling import SMOTE
from collections import Counter

# valence
# вказуємо бажаний баланс (наприклад, збільшуємо мінорні класи до 50 зразків)
sampling_strategy = {cls: max(50, count) for cls, count in Counter(y_train_valence).items()}

smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_train_valence, y_train_valence = smote.fit_resample(X_train_valence, y_train_valence)

print("Після SMOTE:", Counter(y_train_valence))

# arousal
# вказуємо бажаний баланс (наприклад, збільшуємо мінорні класи до 50 зразків)
sampling_strategy = {cls: max(50, count) for cls, count in Counter(y_train_arousal).items()}

smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_train_arousal, y_train_arousal = smote.fit_resample(X_train_arousal, y_train_arousal)

print("Після SMOTE:", Counter(y_train_arousal))

# не вплинуло на точність
# emotion
# вказуємо бажаний баланс (наприклад, збільшуємо мінорні класи до 50 зразків)
sampling_strategy = {cls: max(50, count) for cls, count in Counter(y_train_emotion).items()}

smote = SMOTE(sampling_strategy=sampling_strategy, random_state=42)
X_train_emotion, y_train_emotion = smote.fit_resample(X_train_emotion, y_train_emotion)

print("Після SMOTE:", Counter(y_train_emotion))

ValueError: The {'valence'} target class is/are not present in the data.

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [1000, 1002, 1005, 1007],
    'max_depth': [10, 20, None],
    'criterion': ['gini', 'entropy', 'log_loss']
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [42, 100, 50, 58],
    'oob_score': [True, False],
    'class_weight': [None, "balanced"]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_a_b, y_train_a_b)

print("Найкращі параметри:", grid_search.best_params_)

In [ ]:
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.ensemble import ExtraTreesClassifier
from tqdm import tqdm
import numpy as np
import pandas as pd

# Якщо потрібно, перетворюємо y в масив
y_train_valence = np.array(y_train_valence)

param_list = list(ParameterGrid(param_grid))
print(f"Всього комбінацій: {len(param_list)}")

results = []
for params in tqdm(param_list, desc="GridSearch", unit="комб"):
    model = ExtraTreesClassifier(**params)
    scores = []
    skf = StratifiedKFold(n_splits=3)

    for train_idx, test_idx in skf.split(X_train_valence, y_train_valence):
        X_train_split = X_train_valence.iloc[train_idx] if hasattr(X_train_valence, 'iloc') else X_train_valence[train_idx]
        y_train_split = y_train_valence[train_idx]
        X_test_split = X_train_valence.iloc[test_idx] if hasattr(X_train_valence, 'iloc') else X_train_valence[test_idx]
        y_test_split = y_train_valence[test_idx]

        model.fit(X_train_split, y_train_split)
        score = model.score(X_test_split, y_test_split)
        scores.append(score)

    avg_score = np.mean(scores)
    results.append((params, avg_score))

    # ✅ Виводимо точність для кожної комбінації
    print(f"Параметри: {params} | Точність: {avg_score:.4f}")

# Пошук найкращої комбінації
best_params, best_score = max(results, key=lambda x: x[1])
print("\nНайкращі параметри:", best_params)
print("Найвища середня точність:", best_score)


Всього комбінацій: 2592


GridSearch:   0%|                                                                 | 1/2592 [00:03<2:30:57,  3.50s/комб]

Параметри: {'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 42} | Точність: 0.1945


GridSearch:   0%|                                                                 | 2/2592 [00:07<2:33:02,  3.55s/комб]

Параметри: {'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000, 'random_state': 100} | Точність: 0.1908


...

Параметри: {'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1007, 'random_state': 100} | Точність: 0.1724


GridSearch: 100%|█████████████████████████████████████████████████████████████▉| 2591/2592 [5:39:27<00:05,  5.95s/комб]

Параметри: {'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1007, 'random_state': 50} | Точність: 0.1816


GridSearch: 100%|██████████████████████████████████████████████████████████████| 2592/2592 [5:39:32<00:00,  7.86s/комб]

Параметри: {'class_weight': 'balanced', 'criterion': 'log_loss', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 1007, 'random_state': 58} | Точність: 0.1633

Найкращі параметри: {'class_weight': None, 'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 1005, 'random_state': 50}
Найвища середня точність: 0.20545200655697893


In [ ]:
# навчання моделі
ex_model_v = ExtraTreesClassifier(n_estimators=1005,
                                    criterion='entropy',
                                    max_depth=None,
                                    min_samples_leaf=1,
                                    min_samples_split=5,
                                    # max_features='sqrt',
                                    max_leaf_nodes=None,
                                    #oob_score=True,
                                    random_state=50,
                                    class_weight=None
                                    )
ex_model_v.fit(X_train_valence, y_train_valence)

ExtraTreesClassifier(criterion='entropy', min_samples_split=5,
                     n_estimators=1005, random_state=50)

In [ ]:
y_pred_valence = ex_model_v.predict(X_test_valence)

In [ ]:
print("\nVALENCE\nextratrees\n")
evaluate_model(y_test_valence, y_pred_valence)


VALENCE
extratrees

Accuracy: 0.20437956204379562
Classification Report:
               precision    recall  f1-score   support

           1     0.1818    0.2222    0.2000         9
           2     0.1579    0.1304    0.1429        23
           3     0.2381    0.3448    0.2817        29
           4     0.2381    0.4000    0.2985        25
           5     0.1500    0.1429    0.1463        21
           6     0.0000    0.0000    0.0000        12
           7     0.0000    0.0000    0.0000         7
           8     0.0000    0.0000    0.0000        10
           9     0.0000    0.0000    0.0000         1

    accuracy                         0.2044       137
   macro avg     0.1073    0.1378    0.1188       137
weighted avg     0.1553    0.2044    0.1737       137



In [ ]:
ex_poch_model_v = ExtraTreesClassifier(n_estimators=1000,
                                    criterion='gini',
                                    max_depth=20,
                                    min_samples_leaf=1,
                                    min_samples_split=2,
                                    # max_features='sqrt',
                                    # max_leaf_nodes=None,
                                    # oob_score=True,
                                    random_state=50,
                                    class_weight=None
                                    )
ex_poch_model_v.fit(X_train_valence, y_train_valence)

y_pred_valence = ex_poch_model_v.predict(X_test_valence)
print("\nVALENCE\nextratrees poch\n")
evaluate_model(y_test_valence, y_pred_valence)


VALENCE
extratrees poch

Accuracy: 0.22627737226277372
Classification Report:
               precision    recall  f1-score   support

           1     0.1818    0.2222    0.2000         9
           2     0.2143    0.1304    0.1622        23
           3     0.2727    0.4138    0.3288        29
           4     0.2619    0.4400    0.3284        25
           5     0.1304    0.1429    0.1364        21
           6     0.0000    0.0000    0.0000        12
           7     0.0000    0.0000    0.0000         7
           8     0.0000    0.0000    0.0000        10
           9     0.0000    0.0000    0.0000         1

    accuracy                         0.2263       137
   macro avg     0.1179    0.1499    0.1284       137
weighted avg     0.1734    0.2263    0.1908       137



In [ ]:
# навчання моделі
gb_model_v = GradientBoostingClassifier(n_estimators=1000,
                                    criterion='gini',
                                    max_depth=20,
                                    min_samples_leaf=1,
                                    min_samples_split=2,
                                    max_features='sqrt',
                                    max_leaf_nodes=None,
                                    #oob_score=True,
                                    random_state=50,
                                    #class_weight=None
                                    )
gb_model_v.fit(X_train_valence, y_train_valence)

GradientBoostingClassifier(max_depth=20, max_features='sqrt', n_estimators=1000,
                           random_state=50)

In [ ]:
y_pred_valence = gb_model_v.predict(X_test_valence)

In [ ]:
print("\nVALENCE\n")
evaluate_model(y_test_valence, y_pred_valence)


VALENCE

Accuracy: 0.1897810218978102
Classification Report:
               precision    recall  f1-score   support

           1     0.1818    0.2222    0.2000         9
           2     0.1818    0.1739    0.1778        23
           3     0.2564    0.3448    0.2941        29
           4     0.1667    0.2400    0.1967        25
           5     0.1667    0.1429    0.1538        21
           6     0.0000    0.0000    0.0000        12
           7     0.0000    0.0000    0.0000         7
           8     0.5000    0.1000    0.1667        10
           9     0.0000    0.0000    0.0000         1

    accuracy                         0.1898       137
   macro avg     0.1615    0.1360    0.1321       137
weighted avg     0.1892    0.1898    0.1769       137



In [ ]:
# навчання моделі
rf_model_a = RandomForestClassifier(n_estimators=1000,
                                    criterion='gini',
                                    max_depth=20,
                                    min_samples_leaf=1,
                                    min_samples_split=2,
                                    max_features='sqrt',
                                    max_leaf_nodes=None,
                                    oob_score=True,
                                    random_state=50,
                                    class_weight=None
                                    )
rf_model_a.fit(X_train_arousal, y_train_arousal)

C:\Users\annbu\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=20, n_estimators=1000, oob_score=True,
                       random_state=50)

In [ ]:
y_pred_arousal = rf_model_a.predict(X_test_arousal)

In [ ]:
print("\nAROUSAL\n")
evaluate_model(y_test_arousal, y_pred_arousal)


AROUSAL

Accuracy: 0.18248175182481752
Classification Report:
               precision    recall  f1-score   support

           1     0.5000    0.1429    0.2222         7
           2     0.3125    0.2500    0.2778        20
           3     0.1500    0.1500    0.1500        20
           4     0.0000    0.0000    0.0000        12
           5     0.0000    0.0000    0.0000        17
           6     0.1667    0.3600    0.2278        25
           7     0.2222    0.2609    0.2400        23
           8     0.1667    0.0833    0.1111        12
           9     0.0000    0.0000    0.0000         1

    accuracy                         0.1825       137
   macro avg     0.1687    0.1386    0.1366       137
weighted avg     0.1754    0.1825    0.1654       137



In [ ]:
# навчання моделі
rf_model_e = RandomForestClassifier(n_estimators=1000,
                                    criterion='gini',
                                    max_depth=20,
                                    min_samples_leaf=1,
                                    min_samples_split=2,
                                    max_features='sqrt',
                                    max_leaf_nodes=None,
                                    oob_score=True,
                                    random_state=50,
                                    class_weight=None
                                    )
rf_model_e.fit(X_train_emotion, y_train_emotion)

C:\Users\annbu\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestClassifier(max_depth=20, n_estimators=1000, oob_score=True,
                       random_state=50)

In [ ]:
y_pred_emotion = rf_model_e.predict(X_test_emotion)

In [ ]:
print("\nEMOTION\n")
evaluate_model(y_test_emotion, y_pred_emotion)


EMOTION

Accuracy: 0.2116788321167883
Classification Report:
               precision    recall  f1-score   support

       angry     0.0000    0.0000    0.0000        14
     disgust     0.1600    0.1739    0.1667        23
        fear     0.1538    0.1053    0.1250        19
       happy     0.1429    0.0588    0.0833        17
     neutral     0.2917    0.2500    0.2692        28
         sad     0.2459    0.6000    0.3488        25
    surprise     0.0000    0.0000    0.0000        11

    accuracy                         0.2117       137
   macro avg     0.1420    0.1697    0.1419       137
weighted avg     0.1704    0.2117    0.1743       137



In [ ]:
print("\nVALENCE\n")
evaluate_model(y_test_valence, y_pred_valence)
print("\nAROUSAL\n")
evaluate_model(y_test_arousal, y_pred_arousal)
print("\nEMOTION\n")
evaluate_model(y_test_emotion, y_pred_emotion)


VALENCE

Accuracy: 0.1897810218978102
Classification Report:
               precision    recall  f1-score   support

           1     0.1818    0.2222    0.2000         9
           2     0.1429    0.1304    0.1364        23
           3     0.2500    0.3448    0.2899        29
           4     0.1905    0.3200    0.2388        25
           5     0.1429    0.1429    0.1429        21
           6     0.0000    0.0000    0.0000        12
           7     0.0000    0.0000    0.0000         7
           8     0.0000    0.0000    0.0000        10
           9     0.0000    0.0000    0.0000         1

    accuracy                         0.1898       137
   macro avg     0.1009    0.1289    0.1120       137
weighted avg     0.1455    0.1898    0.1629       137


AROUSAL

Accuracy: 0.18248175182481752
Classification Report:
               precision    recall  f1-score   support

           1     0.5000    0.1429    0.2222         7
           2     0.3125    0.2500    0.2778        20
     

In [ ]:
def evaluate_model(y_true, y_pred, average='micro'):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    # print("Precision:", precision_score(y_true, y_pred, average=average, zero_division=0))
    # print("Recall:", recall_score(y_true, y_pred, average=average, zero_division=0))
    # print("F1 Score:", f1_score(y_true, y_pred, average=average, zero_division=0))
    print("Classification Report:\n", classification_report(y_true, y_pred, digits=4, zero_division=0))

In [ ]:
# підбір гіперпараметрів

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [1000, 1002, 1005, 1007],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'random_state': [42, 100, 50, 58],
    'oob_score': [True, False],
    'class_weight': [None, "balanced"]
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3, scoring='accuracy')
grid_search.fit(X_train_a_b, y_train_a_b)

print("Найкращі параметри:", grid_search.best_params_)

NameError: name 'X_train_a_b' is not defined